In [ ]:
!pip install gym[atari,accept-rom-license]
!pip install moviepy
!pip install pysdl2
!pip install pyvirtualdisplay

In [1]:
# https://github.com/pytorch/examples/blob/master/reinforcement_learning/actor_critic.py
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical



In [2]:
# Cart Pole

seed = 100
gamma = 0.99
render = 'store_true'
log_interval = 10


env = gym.make('CartPole-v1')
env.reset(seed=seed)
torch.manual_seed(seed)


SavedAction = namedtuple('SavedAction', ['log_prob'])


class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

        # policy layer
        self.action_head = nn.Linear(128, 2)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)


        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob


model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float()
    probs = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action)))

    # the action to take (left or right)
    return action.item()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, ), R in zip(saved_actions, returns):

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * R)


    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses
    loss = torch.stack(policy_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]


def main():
    running_reward = 10

    # run infinitely many episodes
    for i_episode in count(1):

        # reset environment and episode reward
        state, _ = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _, _ = env.step(action)

            if render:
                env.render()

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break


main()

/opt/conda/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


Episode 10	Last reward: 18.00	Average reward: 12.26
Episode 20	Last reward: 15.00	Average reward: 12.92
Episode 30	Last reward: 11.00	Average reward: 13.05
Episode 40	Last reward: 12.00	Average reward: 11.96
Episode 50	Last reward: 9.00	Average reward: 12.15
Episode 60	Last reward: 17.00	Average reward: 16.94
Episode 70	Last reward: 8.00	Average reward: 15.70
Episode 80	Last reward: 10.00	Average reward: 13.70
Episode 90	Last reward: 12.00	Average reward: 12.71
Episode 100	Last reward: 17.00	Average reward: 12.20
Episode 110	Last reward: 10.00	Average reward: 11.53
Episode 120	Last reward: 11.00	Average reward: 11.97
Episode 130	Last reward: 19.00	Average reward: 12.65
Episode 140	Last reward: 45.00	Average reward: 15.18
Episode 150	Last reward: 25.00	Average reward: 20.25
Episode 160	Last reward: 22.00	Average reward: 21.45
Episode 170	Last reward: 60.00	Average reward: 27.60
Episode 180	Last reward: 64.00	Average reward: 46.22
Episode 190	Last reward: 115.00	Average reward: 77.45
Epi